# Generate the data for the ISDM-24 track

In [ ]:
%pip install -q pandas openpyxl jinja2

In [7]:
# Print formatted dates in spanish locale
import locale
locale.setlocale(locale.LC_ALL, 'es_ES.UTF-8')

'es_ES.UTF-8'

In [8]:
# Read the dataframe from the .xlsx file given by Easychair
import pandas as pd
df = pd.read_excel('../info/ISDM2024.xlsx',index_col=0,header=0)

# df.Author to spanish 'and' -> 'y'
df['Authors'] = df['Authors'].str.replace(' and ', ' y ', regex=False)
df

,Authors,Title,Tipo,Abstract,Keywords
#,,,,,
2,"Javier Cámara, Javier Troya, Lola Burgueño y A...",On the Assessment of Generative AI in Modeling...,relevante,Most experts agree that Large Language Models ...,"Large Language Models, ChatGPT, Software model..."
3,"Meriem Ben Chaaben, Lola Burgueño y Houari Sah...",Towards using Few-Shot Prompt Learning for Aut...,relevante,We propose a simple yet a novel approach to im...,"Language models, few-shot learning, prompt lea..."
4,"Meysam Karimi, Shekoufeh Kolahdouz Rahimi y Ja...",Ant-colony optimization for automating test mo...,relevante,"In model transformation (MT) testing, test dat...","Model-Driven Engineering, Model Transformation..."
12,"Cesar Gonzalez Mora, Cristina Barros, Irene Ga...",Improving Open Data Web API Documentation thro...,relevante,Widely adoption of Information Technologies ha...,"Web API, OpenAPI documentation, Natural Langua..."
24,Pedro Juan Molina Moreno y Javier Centeno Vega,Orca: Nocode Graphical Editor for Container Or...,corto,Traditional Container Orchestration tools like...,"MDE, DSL, nocode, containers, graphical editor..."
57,"Rubén Salado-Cid, Antonio Vallecillo, Kamran M...",SWEL: A Domain-Specific Language for Modeling ...,relevante,Data-intensive applications aim at discovering...,"Model-driven engineering, Domain-specific mode..."
81,"Guillermo Fuentes, Francisco Ruiz y Angelica Caro",Modelado dirigido por las preocupaciones de lo...,herramienta,Un punto de vista contiene un conjunto de acue...,"Arquitectura Empresarial, Archi, ArchiMate, Pu..."
91,"Abel Gómez, Iván Alfonso, Silvia Doñate, Jordi...",Role-based access control in wastewater treatm...,completo,Wastewater treatment plants (WWTPs) are critic...,"Model-driven engineering (MDE), Role-based acc..."
94,"José Ángel Barriga Corchero, Pedro José Clemen...","Design, code generation and simulation of IoT ...",relevante,The Internet of Things (IoT) is rapidly spread...,"IIoT systems, IoT simulation, IoT mobility, Mo..."


In [29]:
from dataclasses import dataclass
from datetime import datetime, timedelta

@dataclass
class Session:
    name: str
    start: datetime
    paperstart: datetime
    end: datetime
    location: str
    chair: str
    papers: list[int]

isdm1 = Session('ISDM-1',
                datetime(2024,7,18,9,15),
                datetime(2024,7,18,9,15)+timedelta(minutes=10), # 10 mins presentation
                datetime(2024,7,18,10,15),
                'Aula A.3.1',
                'Diego Sevilla',
                [91, 94, 24])

isdm2 = Session('ISDM-2',
                datetime(2024,7,18,16,00),
                datetime(2024,7,18,16,00),
                datetime(2024,7,18,17,30),
                'Aula A.3.1',
                'Pedro J. Clemente',
                [3, 2, 4, 121, 124])

isdm3 = Session('ISDM-3',
                datetime(2024,7,18,18,00),
                datetime(2024,7,18,18,00),
                datetime(2024,7,18,19,00),
                'Aula A.3.1',
                'Diego Sevilla y Pedro J. Clemente',
                [12, 57, 81])

sessions = {s.name : s for s in [isdm1, isdm2, isdm3]}

# No article left behind
assert set(df.index) == set().union(*list(map(lambda s: s.papers, sessions.values())))

In [30]:
# Produce the slide index from the slideindex template
from jinja2 import Template

def gen_session_index(sessions: dict[str, Session]) -> None:
    template = Template(open('templates/sessionindex.tex.j2').read())
    with open('../pr/sessionindex.tex', 'w') as f:
        f.write(template.render(sessions=sessions))

gen_session_index(sessions)

In [32]:
def gen_session_slides(sessions: dict[str, Session], df: pd.DataFrame) -> None:
    template = Template(open('templates/sessionslide.tex.j2').read())

    def get_duration_time(articletype: str):
        match articletype:
            case 'relevante' | 'completo':
                return 20
            case 'corto' | 'herramienta':
                return 10
            case _:
                raise ValueError(f'Unknown article type {articletype}')

    def get_duration(articletype: str):
        match articletype:
            case 'relevante' | 'completo':
                return f'{get_duration_time(articletype)}~min'
            case 'corto' | 'herramienta':
                return f'{get_duration_time(articletype)}~min'
            case _:
                raise ValueError(f'Unknown article type {articletype}')

    def time_str(d: datetime) -> str:
        return d.strftime('%H:%M')

    def date_str(d: datetime) -> str:
        return d.strftime('%A, %d de %B de %Y, %H:%M')

    for session_name, session in sessions.items():
        with open(f'../pr/{session_name}.tex', 'w') as f:
            f.write(template.render(session=session,
                                    papers=session.papers,
                                    date_str = date_str,
                                    time_str = time_str,
                                    df=df,
                                    get_duration=get_duration,
                                    get_duration_time=get_duration_time,
                                    timedelta=timedelta))

gen_session_slides(sessions, df)